In [2]:
import subprocess
import lizard
import json
import shutil
from openai import OpenAI

In [20]:
def create_copy_of_project(file_path):
  ignore_patterns = shutil.ignore_patterns('.*')
  return shutil.copytree(file_path, file_path + '-copy', dirs_exist_ok=True, ignore=ignore_patterns)

In [31]:
def measure_test_coverage(project_dir):
  try:
    subprocess.run(['cd ' + project_dir + ' && npx nyc --exclude examples --exclude test --exclude benchmarks --reporter=json-summary npm test'], shell=True, capture_output=True, text=True, check=True)
    with open(project_dir + '/coverage/coverage-summary.json', "r") as coverage_summary: 
      coverage_info = json.load(coverage_summary)
    
    coverage_info_cleansed = dict()
    for module in coverage_info:
      coverage_info_cleansed[module.replace(project_dir, '')] = {'coverage': coverage_info[module]}
      
    return coverage_info_cleansed
    
  except subprocess.CalledProcessError as e:
    print(f"An error occurred: {e}")
    return None  

In [39]:
def compute_cyclomatic_complexity(project_dir, code_dir):
  analysis = lizard.analyze(paths=[project_dir + code_dir])
  
  functions = list()
  for file in list(analysis):
    for function in file.function_list:
      functions.append(function)
  
  return functions

In [53]:
def find_most_complex_function(functions):
  result = sorted(functions, key=lambda fun: fun.cyclomatic_complexity, reverse=True)
  return result[0]

In [54]:
def main():
  # Measure test coverage before changes
  # Measure complexity of code files
  # Have LLM refactor file
  # see if tests pass
  # Measure test coverage after changes
  # Measure complexity after changes
  # Provide summary of how values have changed

  project_dir = '/media/lebkuchen/ba3cb2a9-fcaa-4daf-8c11-e1006cc10771/Repos/express'
  code_dir = '/lib'

  #coverage_info = measure_test_coverage(project_dir)
  complexity_info = compute_cyclomatic_complexity(project_dir, code_dir)
  most_complex = find_most_complex_function(complexity_info)
  
  # project_copy_dir = create_copy_of_project(project_dir)
  
if __name__ == "__main__":
    main()


{'cyclomatic_complexity': 24, 'nloc': 73, 'token_count': 425, 'name': 'send', 'long_name': 'send ( body )', 'start_line': 114, 'end_line': 214, 'full_parameters': ['body'], 'filename': '/media/lebkuchen/ba3cb2a9-fcaa-4daf-8c11-e1006cc10771/Repos/express/lib/response.js', 'top_nesting_level': 0, 'fan_in': 0, 'fan_out': 0, 'general_fan_out': 0}


In [56]:
key_file = open('openai-key.txt', "r")
api_key = key_file.read() 
client = OpenAI(api_key=api_key)

file_dir = '/media/lebkuchen/ba3cb2a9-fcaa-4daf-8c11-e1006cc10771/Repos/express/lib/view.js'

with open(file_dir, "r") as code_file:
   view = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {"role": "system", "content": "Refactor the provided javascript source code file in a way that its cyclomatic complexity is reduced"},
      {"role": "user", "content": code_file.read()}
    ]
  )